In [1]:
%matplotlib inline

In [2]:
import datetime
import pandas as pd
import numpy as np
import datetime

In [3]:
import catboost
from catboost import CatBoostClassifier, Pool, cv
import hyperopt

In [4]:
import tqdm

In [5]:
from sklearn.pipeline import Pipeline
import sklearn.model_selection
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn import model_selection , metrics   #Additional scklearn functions

## import and parameters

In [6]:
imput_strategy = 3

In [7]:
train = pd.read_pickle('../data/merged_data/train.pkl')
test = pd.read_pickle('../data/merged_data/test.pkl')

In [8]:
#train = train.sample(frac=0.1)

In [9]:
# preprocess
y_train = train['target'].values

# to try:
* optimize aucroc with keras and batch methods
* SVM / rank SVM (https://github.com/rdipietro/pyrvm)
* try separate models on separate variables + stacking

## Features : 
* NLP on COMMENTAIRE_BI
* Extract options from OPTION (pas sur que ce soit util, peut être juste check la qualité)
* features from history



* check for intervention in test without contracts and to handle them
* add combinations of categorical features (see paribas example on catboost site)
* select only best features (feature_importance ?)
* try to put random nas in train for handling missing contracts

## Fails:
* one model with selectKbest on modalities vs one model on dimensionality reduction with group rare modalities
    * dimensionality reduction don't bring much value here (mb MCA)
    * select k best fails with 5k categories, which is not enough to be interesting

In [10]:
drop = ['INSTANCE_ID', #460k modalities, not usable as a feature
        'INCIDENT_NUMBER']
drop_atm = [#'AUTEUR_INCIDENT', # 2088 modalities
            'TYPE_VOIE',
#            'NATURE_CODE', # 313 modalities, need to be splitted in 5 modalities
#            'MARQUE_LIB', # 167 modalities
#            'OPTION', # 80 modalities, extract options
#            'MODELE_CODE', # 10k modalities
#            'COMMENTAIRE_BI', # NLP 400k modalities
#             'RESOURCE_ID', # 4033 modalities
            'CODE_POSTAL', # 5800 modalities (only get first 2 numbers ?)
            'L2_ORGA_CODE_POSTAL', # 147 modalities (might be redondent with L2_ORGANISATION_ID)
#            'L2_ORGANISATION_ID' #151 modalities
            'L2_ORGA_VILLE', # 146, might be redondent with other organisation variables
#            'RACHAT_CODE' # 312 modalities (try binarising ?)         
#            'CODE_INSTALLATION' # 17 modalities
           ]

In [11]:
train.drop(drop + drop_atm + ['target'], axis=1, inplace=True)
test.drop(drop + drop_atm, axis=1, inplace=True)

In [12]:
categoricals = list(train.columns[train.dtypes == 'category'])

### imputation of missing data

TODO: try imputing test based on test values, not train <br>
TODO: try diffrent strategy on imputing datas from contract since missing are present only in test set<BR>
TODO: try creating data with missing contract in train sample and do not fill the missing in test<br>
TODO: try imputing specific na value for categorical variables

In [13]:
def impute_selected_variables(df, test, categ, quanti, dates):
    _df = df.copy()
    _test = test.copy() if test is not None else None
    
    replace = _df[categ].mode()
    replace_values = {k:v.iloc[0] for k,v in replace.items()}
    _df.fillna(replace_values, inplace=True)

    replace_quanti = _df[quanti].mean()
    _df.fillna(replace_quanti, inplace=True)

    _df[dates] = _df[dates].fillna(method='pad')
    
    if test is not None:
        
        _test.fillna(replace_values, inplace=True)
        _test.fillna(replace_quanti, inplace=True)
        _test[dates] = _df[dates].fillna(method='pad')
    
    return _df, _test

In [14]:
#replace = train[categoricals].mode()
#replace_values = {k:v.iloc[0] for k,v in replace.items()}
def impute_contract_variables(df):
    _df = df.copy()
    
    for var in categ_contract:
        _df[var] = _df[var].cat.add_categories(['NAN'])
        _df[var].fillna('NAN', inplace=True)

    _df[quanti_contract] = _df[quanti_contract].fillna(-9999)
    _df[date_contract] = _df[date_contract].fillna(datetime.datetime(1970, 1, 1))
    return _df

In [15]:
categoricals = list(train.columns[train.dtypes == 'category'])
quantitative = ['NB_PASSAGE', 'POINTS_FIDEL', 'CONTRAT_TARIF', 'PRIX_FACTURE']
dates = list(train.columns[train.dtypes == 'datetime64[ns]'])

In [16]:
contract_variables = [ 'UPD_DATE', 'DATE_DEBUT', 'DATE_FIN', 'STS_CODE', 'OPTION', 'FORMULE', 'CONTRAT_TARIF', 'PRIX_FACTURE']

In [17]:
#impute without contract
categ_to_impute = list(set(categoricals) - set(contract_variables))
quanti_to_impute = list(set(quantitative) - set(contract_variables))
date_to_impute = list(set(dates) - set(contract_variables))

In [18]:
#impute contract
categ_contract = list(set(categoricals).intersection(set(contract_variables)))
quanti_contract = list(set(quantitative).intersection(set(contract_variables)))
date_contract = list(set(dates).intersection(set(contract_variables)))

In [19]:
#train and test are filled with values taken from train
#contract and other variables are imputed together
if imput_strategy == 1:
    train, test = impute_selected_variables(train, test, categoricals, quantitative, dates)

In [20]:
#train and test are filled with values taken on their own values
#contract and other variables are imputed together
if imput_strategy == 2:
    train, _ = impute_selected_variables(train, None, categoricals, quantitative, dates)
    test, _ = impute_selected_variables(test, None, categoricals, quantitative, dates)

In [21]:
#train and test are filled with values taken from train
#contract and other variables are imputed separatly (need to import some NAN in train set)
if imput_strategy == 3:
    train, test = impute_selected_variables(train, test, categ_to_impute, quanti_to_impute, date_to_impute)
    train = impute_contract_variables(train)
    test = impute_contract_variables(test)

In [22]:
# train and test are filled on their own values
# contract and other variables are imputed separatly (need to import some NAN in train set)
if imput_strategy == 4:
    train, _ = impute_selected_variables(train, None, categ_to_impute, quanti_to_impute, date_to_impute)
    test, _ = impute_selected_variables(test, None, categ_to_impute, quanti_to_impute, date_to_impute)
    train = impute_contract_variables(train)
    test = impute_contract_variables(test)

### Feature ingineering

In [23]:
def commentaire_bi(df):
    _df = df.copy()
    
    _df.COMMENTAIRE_BI = _df.COMMENTAIRE_BI.str.upper()
    COMMENTAIRE_BI_vc = _df.COMMENTAIRE_BI.value_counts()
    common_commentaire_bi = COMMENTAIRE_BI_vc[COMMENTAIRE_BI_vc > 100].index
    _df['COMMENTAIRE_BI_common'] = _df.COMMENTAIRE_BI.where(_df.COMMENTAIRE_BI.isin(common_commentaire_bi), "Rare")
    
    _df['nb_char_commentaire'] = [len(txt) for txt in _df.COMMENTAIRE_BI]
    _df['nb_mots_commentaire'] = [len(txt.split()) for txt in _df.COMMENTAIRE_BI]
    _df['has_number_commentaire'] = [any(char.isdigit() for char in txt) for txt in _df.COMMENTAIRE_BI]
    _df['is_empty_commentaire'] = [(txt == '.') for txt in _df.COMMENTAIRE_BI]
    _df.drop('COMMENTAIRE_BI', axis=1, inplace=True)
    
    return _df

In [24]:
def nature_code_split(df):
    _df = df.copy()
    nature_code_splitted = [nc.split('-') for nc in df.NATURE_CODE]
    nature_code_df = pd.DataFrame(nature_code_splitted, columns=['nc_1', 'nc_2', 'nc_3', 'nc_4', 'nc_5'])
    nature_code_df.fillna('-1', inplace=True)
    for nc_i in ['nc_1', 'nc_2', 'nc_3', 'nc_4', 'nc_5']:
        nature_code_df[nc_i] = nature_code_df[nc_i].astype('category')
    
    #_df.drop('NATURE_CODE', axis=1, inplace=True)
    _df = _df.merge(nature_code_df, left_index=True, right_index=True)
    return _df

In [25]:
# todo: use dt series accessor
def add_dates_features(data):
    data['age_installation'] = (data['CRE_DATE_GZL'] - data['INSTALL_DATE']).dt.days // 365
    data['mois_appel'] = data['CRE_DATE_GZL'].map(lambda x: x.month)
    data['joursemaine_appel'] = data['CRE_DATE_GZL'].map(lambda x: x.isoweekday()) #integer, might be considered categorical
    data['jour_appel'] = data['CRE_DATE_GZL'].map(lambda x: x.day)
    data['mois_intervention'] = data['SCHEDULED_START_DATE'].map(lambda x: x.month)
    data['joursemaine_intervention'] = data['SCHEDULED_START_DATE'].map(lambda x: x.isoweekday()) #integer, might be considered categorical
    data['jour_intervention'] = data['SCHEDULED_START_DATE'].map(lambda x: x.day)
    data['duree_avant_intervention'] = (data['SCHEDULED_START_DATE'] - data['CRE_DATE_GZL']).dt.days
    data['duree_prevue'] = (data['SCHEDULED_END_DATE'] - data['SCHEDULED_START_DATE']).dt.days
    data['temps_depuis_debut_contrat'] = (data['CRE_DATE_GZL'] - data['DATE_DEBUT']).dt.days
    data['temps_jusqua_fin_contrat'] = (data['CRE_DATE_GZL'] - data['DATE_FIN']).dt.days  #souvent nan ? (mettre 0)
    data['temps_depuis_maj_contrat'] = (data['CRE_DATE_GZL'] - data['UPD_DATE']).dt.days 

    data.drop(['CRE_DATE_GZL', 'INSTALL_DATE', 'SCHEDULED_START_DATE', 'SCHEDULED_END_DATE', 'DATE_DEBUT', 'DATE_FIN', 'UPD_DATE'], axis=1, inplace=True)
    return data

In [26]:
#mots en particulier
#créer un dictionnaire et compter (count et garder les must ? voir la diff entre les cas + et - ?)

In [27]:
# feature engineering

# (history) temps depuis dernière visite (pas forcément dispo sur le test)
# (history) déjà eu une casse sur ce matériel
# (history) temps depuis dernière casse
# (history) la dernière visite date de moins de 6 mois
# (history) nb interventions faires par la ressource
# (history) temps depuis la première intervention de la ressource
# (contract history) nb de fois que le contrat a été mis à jour sur les X dernières années

In [28]:
train = commentaire_bi(train)
train = nature_code_split(train)
train = add_dates_features(train)

In [29]:
test = commentaire_bi(test)
test = nature_code_split(test)
test = add_dates_features(test)

In [30]:
#drop variables
drop = ['joursemaine_appel', 
'USAGE_LOCAL', 
'nc_4', 
'is_empty_commentaire', 
'duree_prevue', 
'nc_1']

train.drop(drop, axis=1,inplace=True)
test.drop(drop, axis=1,inplace=True)

## Get variables where test set has modalities which are not in train set

In [64]:
categoricals = list(train.columns[train.dtypes == 'category'])
var_with_new_categ = []
for cat in categoricals:
    if len(set(test[cat]) - set(train[cat])) > 0:
        var_with_new_categ.append(cat)

# drop categories in test which are not in train

In [60]:
for cat in categoricals:
    train[cat] = train[cat].cat.remove_unused_categories()

AttributeError: Can only use .cat accessor with a 'category' dtype

In [89]:
class CategoriesDroper(BaseEstimator, TransformerMixin):
    '''Drop categories which are not in the train set'''

    def __init__(self, columns):
        self.columns = columns
        self.categories_dict = dict()
        
    def fit(self, df, y=None):
        self.categories_dict = {column: df[column].cat.categories for column in self.columns}
        return self

    def transform(self, df):
        _df = df.copy()
        for column in self.columns:
            _df[column] = _df[column].cat.set_categories(self.categories_dict[column])
            try:
                _df[column] = _df[column].fillna('NAN')
            except ValueError as e:
                _df[column] = _df[column].cat.add_categories(['NAN'])
                _df[column] = _df[column].fillna('NAN') 
                print(e,'with ', column)
        
        return _df

In [90]:
cd = CategoriesDroper(categoricals)
cd.fit(train)
test = cd.transform(test)

fill value must be in categories in  INCIDENT_TYPE_ID
fill value must be in categories in  TYPE_BI
fill value must be in categories in  MILLESIME
fill value must be in categories in  PROBLEM_CODE
fill value must be in categories in  AUTEUR_INCIDENT
fill value must be in categories in  GRAVITE
fill value must be in categories in  RESOURCE_ID
fill value must be in categories in  TYPE_OCC
fill value must be in categories in  RACHAT_CODE
fill value must be in categories in  NATURE_CODE
fill value must be in categories in  MARQUE_LIB
fill value must be in categories in  MODELE_CODE
fill value must be in categories in  PAYS
fill value must be in categories in  STOP_PHONING
fill value must be in categories in  CODE_GEN_EQUIPEMENT
fill value must be in categories in  CODE_FONCTION
fill value must be in categories in  CODE_ENERGIE
fill value must be in categories in  CODE_INSTALLATION
fill value must be in categories in  CODE_SPECIFICATION
fill value must be in categories in  CODE_EAU_CHAUDE
fi

# Modelisation

TODO:
* one hot encoding low modalities: keep for grid search (no ideal value found right now, try 3, 5, ... 9)
* group rare modalities: doesn't seem to improve the model

### split train / val

In [ ]:
#for item in categoricals:
#    train[item] = train[item].cat.codes +1

In [ ]:
# split train in train, cv (will be replaced by cross validation for parameters tuning)
# stratify ?
X_train_train, X_train_val, y_train_train, y_train_val = sklearn.model_selection.train_test_split(train, y_train, random_state=101)

In [ ]:
pos_neg_ratio =  sum(y_train==0) / sum(y_train)

In [ ]:
#combinations_ctr for list of variables ?
model = CatBoostClassifier(
    iterations=200,
#    one_hot_max_size=3,
#    learning_rate=0.16,
    depth=8,
    eval_metric="AUC",
    random_seed=42,
    od_type='Iter',
    od_wait=40,
    use_best_model=True
#    scale_pos_weight=pos_neg_ratio
)

In [ ]:
categoricals = train.select_dtypes(include=['category', 'bool', 'object']).columns
categorical_features_indices = [X_train_train.columns.get_loc(cat) for cat in categoricals]

In [ ]:
model.fit(
    X_train_train, y_train_train,
    cat_features=categorical_features_indices,
    eval_set=(X_train_val, y_train_val),
    logging_level='Verbose'  # you can uncomment this for text output
)

199:	test: 0.7686368	best: 0.7686409 (198)	total: 14m 41s	remaining: 0us (nlp, lr=0.31)
199:	test: 0.7686109	best: 0.7686109 (199)	total: 14m 17s	remaining: 0us (nlp, lr=0.31, 9 variables dropped)
199:	test: 0.7729298	best: 0.7730353 (190)	total: 22m 13s	remaining: 0us (nlp, lr=0.31, 6 variables dropped, depth=8)
199:	test: 0.7576144	best: 0.7576257 (197)	total: 15m 42s	remaining: 0us(nlp, lr=0.31, weighted, one_hot=3)

### Feature importance

In [ ]:
importances=model.feature_importances_

In [ ]:
indices = np.argsort(model.feature_importances_)[::-1]

print("Feature ranking:")

for f in range(train.shape[1]):
    print("%d. %s (%f)" % (f + 1, train.columns[indices[f]], model.feature_importances_[indices[f]]))

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# Plot the feature importances of the forest
plt.figure()
plt.title("Feature importances")
plt.bar(range(train.shape[1]), importances[indices],
       color="r", align="center")
plt.xticks(range(train.shape[1]), indices)
plt.xlim([-1, train.shape[1]])
plt.show()

### Hyperparameter tunning

In [ ]:
categoricals = train.select_dtypes(include=['category', 'bool', 'object']).columns
categorical_features_indices = [train.columns.get_loc(cat) for cat in categoricals]

#### GridSearch

In [ ]:
from sklearn.grid_search import GridSearchCV

In [ ]:
#what to do with cat features
params = {'depth': [4, 7, 10],
          'learning_rate' : [0.20, 0.31, 0.40],
         'l2_leaf_reg': [1, 4, 9],
         'iterations': [300]}
cb = CatBoostClassifier()
cb_model = GridSearchCV(cb, params, scoring="roc_auc")
cb_model.fit(train, y_train, cb__cat_features=categorical_features_indices)

#### hyperopt

In [ ]:
def hyperopt_objective(params):
    model = CatBoostClassifier(
        l2_leaf_reg=int(params['l2_leaf_reg']),
        learning_rate=params['learning_rate'],
        iterations=int(params['iterations']),
        border_count=int(params['border_count']),
        eval_metric='AUC',
        random_seed=42,
        logging_level='Silent'
        #od_type='Iter',
        #od_wait=40
    )
    
    cv_data = cv(
        Pool(train, y_train, cat_features=categorical_features_indices),
        model.get_params(),
        stratified=True,
        fold_count=5
    )
    best_AUC = np.max(cv_data['test-AUC-mean'])
    print(params, 'best_AUC: ', best_AUC)
    
    return 1 - best_AUC # as hyperopt minimises

In [ ]:
#param space example for xgboost
#    space = {
#             'n_estimators' : hp.quniform('n_estimators', 100, 1000, 1),
#             'eta' : hp.quniform('eta', 0.025, 0.5, 0.025),
#             'max_depth' : hp.quniform('max_depth', 1, 13, 1),
#             'min_child_weight' : hp.quniform('min_child_weight', 1, 6, 1),
#             'subsample' : hp.quniform('subsample', 0.5, 1, 0.05),
#             'gamma' : hp.quniform('gamma', 0.5, 1, 0.05),
#             'colsample_bytree' : hp.quniform('colsample_bytree', 0.5, 1, 0.05),
#             'num_class' : 9,
#             'eval_metric': 'mlogloss',
#             'objective': 'multi:softprob',
#             'nthread' : 6,
#             'silent' : 1
#}

In [ ]:
from numpy.random import RandomState

params_space = {
    #'l2_leaf_reg': hyperopt.hp.loguniform('l2_leaf_reg', -1, np.log(50)),
    'l2_leaf_reg': hyperopt.hp.choice('l2_leaf_reg', [11.47871028241772]),
    'learning_rate': hyperopt.hp.uniform('learning_rate', 1e-1, 8e-1),
     #'learning_rate': hyperopt.hp.choice('learning_rate', [0.31]),
    'iterations': hyperopt.hp.quniform('iterations', 250, 1000, 1),
    #'iterations': hyperopt.hp.choice('iterations', [250]),
    #'depth': hyperopt.hp.quniform('depth', 3, 9, 1),
    'depth': hyperopt.hp.choice('depth', [7]),
#    'ctr_border_count': hyperopt.hp.quniform('ctr_border_count', 32, 255, 1),
    #'border_count': hyperopt.hp.quniform('border_count', 16, 255, 1)
    'border_count': hyperopt.hp.choice('border_count', [213])
}

trials = hyperopt.Trials()

best = hyperopt.fmin(
    hyperopt_objective,
    space=params_space,
    algo=hyperopt.tpe.suggest,
    max_evals=8,
    trials=trials,
    rstate=RandomState(123)
    
)

print(best)

#round 1
{'depth': 7.0, 'l2_leaf_reg': 6.687638927479829} best_AUC:  0.764339203919555
{'depth': 8.0, 'l2_leaf_reg': 2.238434810070511} best_AUC:  0.7641940842160905
{'depth': 9.0, 'l2_leaf_reg': 15.61615505112418} best_AUC:  0.7656179177534072
{'depth': 7.0, 'l2_leaf_reg': 11.47871028241772} best_AUC:  0.766724797438951
{'depth': 6.0, 'l2_leaf_reg': 2.0946141383866816} best_AUC:  0.7641940842160905
{'depth': 5.0, 'l2_leaf_reg': 3.409700409045487} best_AUC:  0.7656396435014416
{'l2_leaf_reg': 11.47871028241772, 'learning_rate': 7.0}

#round 2
{'border_count': 199.0, 'depth': 7, 'iterations': 250, 'l2_leaf_reg': 11.47871028241772, 'learning_rate': 0.31} best_AUC:  0.7654693707749893
{'border_count': 213.0, 'depth': 7, 'iterations': 250, 'l2_leaf_reg': 11.47871028241772, 'learning_rate': 0.31} best_AUC:  0.7662068818039395
{'border_count': 105.0, 'depth': 8, 'iterations': 250, 'l2_leaf_reg': 11.47871028241772, 'learning_rate': 0.31} best_AUC:  0.7639026943327258
{'border_count': 238.0, 'depth': 7, 'iterations': 250, 'l2_leaf_reg': 11.47871028241772, 'learning_rate': 0.31} best_AUC:  0.764574925126635
{'border_count': 22.0, 'depth': 8, 'iterations': 250, 'l2_leaf_reg': 11.47871028241772, 'learning_rate': 0.31} best_AUC:  0.7643064612539859
{'border_count': 103.0, 'depth': 8, 'iterations': 250, 'l2_leaf_reg': 11.47871028241772, 'learning_rate': 0.31} best_AUC:  0.7632638136706521
{'border_count': 225.0, 'depth': 8, 'iterations': 250, 'l2_leaf_reg': 11.47871028241772, 'learning_rate': 0.31} best_AUC:  0.7653002122083675
{'border_count': 212.0, 'depth': 8, 'iterations': 250, 'l2_leaf_reg': 11.47871028241772, 'learning_rate': 0.31} best_AUC:  0.7649391813119792
{'border_count': 85.0, 'depth': 8, 'iterations': 250, 'l2_leaf_reg': 11.47871028241772, 'learning_rate': 0.31} best_AUC:  0.7649657537494025
{'border_count': 251.0, 'depth': 8, 'iterations': 250, 'l2_leaf_reg': 11.47871028241772, 'learning_rate': 0.31} best_AUC:  0.7649243873489558
{'border_count': 213.0, 'depth': 0, 'iterations': 0, 'l2_leaf_reg': 0, 'learning_rate': 0}

#round 3
{'border_count': 213, 'depth': 7, 'iterations': 693.0, 'l2_leaf_reg': 11.47871028241772, 'learning_rate': 0.6096210638782021} best_AUC:  0.767539056284941
{'border_count': 213, 'depth': 7, 'iterations': 526.0, 'l2_leaf_reg': 11.47871028241772, 'learning_rate': 0.701085638831484} best_AUC:  0.767228088670764
{'border_count': 213, 'depth': 7, 'iterations': 822.0, 'l2_leaf_reg': 11.47871028241772, 'learning_rate': 0.7947626751796161} best_AUC:  0.7647414699770261
{'border_count': 213, 'depth': 7, 'iterations': 775.0, 'l2_leaf_reg': 11.47871028241772, 'learning_rate': 0.571434077994127} best_AUC:  0.7686875566115616
{'border_count': 213, 'depth': 7, 'iterations': 516.0, 'l2_leaf_reg': 11.47871028241772, 'learning_rate': 0.4938304425560929} best_AUC:  0.7680238623609622


In [ ]:
#check quality to expect
model = CatBoostClassifier(
    l2_leaf_reg=int(best['l2_leaf_reg']),
    learning_rate=best['learning_rate'],
    iterations=500,
    eval_metric='AUC',
    random_seed=42,
    logging_level='Silent'
)
cv_data = cv(Pool(X, y, cat_features=categorical_features_indices), model.get_params())

In [ ]:
print('Precise validation AUC score: {}'.format(np.max(cv_data['test-AUC-mean'])))

### Make submission

In [ ]:
categoricals = train.select_dtypes(include=['category', 'bool', 'object']).columns
categorical_features_indices = [train.columns.get_loc(cat) for cat in categoricals]

In [ ]:
pos_neg_ratio =  sum(y_train==0) / sum(y_train==1)

In [ ]:
params = {'border_count': 213, 
          'one_hot_max_size': 5,
#          'depth': 7,
          'depth': 5,
          'iterations': 200.0, 
          'l2_leaf_reg': 11.47871028241772, 
#          'learning_rate': 0.4938304425560929,
          'scale_pos_weight': pos_neg_ratio
         }

#random_strength (default 1)

In [ ]:
model = CatBoostClassifier(
    **params,
    eval_metric="AUC",
    od_type='Iter',
    od_wait=40
)

model.fit(train, y_train, cat_features=categorical_features_indices)

In [ ]:
#save model to disk
#model.save_model('catboost_model.dump')
#load
#model = CatBoostClassifier()
#model.load_model('catboost_model.dump');

In [ ]:
submisstion = model.predict_proba(test)[:, 1]

In [ ]:
filename = str(datetime.datetime.now())[:-7] + '_submission.csv'
sub = pd.Series(submisstion, name='target')
sub.to_csv(filename, index=False, header=True)

### tests on cross validation

In [ ]:
pos_neg_ratio =  sum(y_train==0) / sum(y_train==1)

categoricals = train.select_dtypes(include=['category', 'bool', 'object']).columns
categorical_features_indices = [train.columns.get_loc(cat) for cat in categoricals]

model = CatBoostClassifier(
    border_count=128, 
    one_hot_max_size= 5,
    depth= 5,
    iterations= 200.0, 
    l2_leaf_reg= 11.478, 
    scale_pos_weight= pos_neg_ratio,
    eval_metric='AUC'
)

In [ ]:
cv_data = cv(
    Pool(train, y_train, cat_features=categorical_features_indices),
    model.get_params(),
    stratified=True,
    fold_count=5
)

#0.707 on test set

In [ ]:
cv_data.plot(y=['test-AUC-mean', 'train-AUC-mean'])

In [ ]:
cv_data.plot(y=['test-Logloss-mean', 'train-Logloss-mean'])

### train with some randomly added na for contract variables

In [44]:
var_with_new_categ_not_in_contract = [var for var in var_with_new_categ if var not in categ_contract]

In [45]:
quanti_contract = ['PRIX_FACTURE', 
                   'CONTRAT_TARIF',
                   'temps_depuis_debut_contrat',
                   'temps_jusqua_fin_contrat', 
                   'temps_depuis_maj_contrat']

In [46]:
for var in categ_contract + var_with_new_categ_not_in_contract:
    try:
        train[var] = train[var].cat.add_categories(['NAN'])
    except ValueError as e:
        next

In [51]:
categoricals = train.select_dtypes(include=['category', 'bool', 'object']).columns
categorical_features_indices = [train.columns.get_loc(cat) for cat in categoricals]

In [53]:
models = []

#np.random.seed(42)
for i in tqdm.tqdm_notebook(range(2)):
    X_train = train.copy()
    
    #randomly put missing contract in train data
    _idx = np.random.choice(X_train.index, size=X_train.shape[0]//20, replace=False)
    X_train.loc[_idx, categ_contract] = 'NAN'
    X_train.loc[_idx, quanti_contract] = -9999
    
    #randomly put missing in variables with new categories in test
    for var in var_with_new_categ_not_in_contract:
        _idx = np.random.choice(X_train.index, size=X_train.shape[0]//20, replace=False)
        X_train.loc[_idx, var] = 'NAN'

    model = CatBoostClassifier(
        iterations=150,
        depth=6,
        border_count=128,
        learning_rate=0.30,
        eval_metric='AUC'
    )
    
    model.fit(X_train, y_train, cat_features=categorical_features_indices)
    models.append(model.copy())

0:	total: 3.39s	remaining: 8m 25s
1:	total: 6.62s	remaining: 8m 9s
2:	total: 10s	remaining: 8m 11s
3:	total: 13.2s	remaining: 8m 3s
4:	total: 16.5s	remaining: 7m 59s
5:	total: 19.5s	remaining: 7m 48s
6:	total: 22.5s	remaining: 7m 39s
7:	total: 25.9s	remaining: 7m 40s
8:	total: 30.2s	remaining: 7m 53s
9:	total: 34.5s	remaining: 8m 2s
10:	total: 38.2s	remaining: 8m 2s
11:	total: 41.7s	remaining: 7m 59s
12:	total: 46s	remaining: 8m 4s
13:	total: 49.1s	remaining: 7m 56s
14:	total: 52s	remaining: 7m 48s
15:	total: 55.1s	remaining: 7m 41s
16:	total: 58.2s	remaining: 7m 35s
17:	total: 1m 1s	remaining: 7m 30s
18:	total: 1m 4s	remaining: 7m 26s
19:	total: 1m 7s	remaining: 7m 21s
20:	total: 1m 11s	remaining: 7m 19s
21:	total: 1m 15s	remaining: 7m 19s
22:	total: 1m 18s	remaining: 7m 14s
23:	total: 1m 21s	remaining: 7m 8s
24:	total: 1m 24s	remaining: 7m 2s
25:	total: 1m 27s	remaining: 6m 58s
26:	total: 1m 30s	remaining: 6m 53s
27:	total: 1m 35s	remaining: 6m 58s
28:	total: 1m 40s	remaining: 6m 57s

82:	total: 5m 32s	remaining: 4m 28s
83:	total: 5m 36s	remaining: 4m 24s
84:	total: 5m 39s	remaining: 4m 19s
85:	total: 5m 43s	remaining: 4m 15s
86:	total: 5m 46s	remaining: 4m 10s
87:	total: 5m 49s	remaining: 4m 6s
88:	total: 5m 53s	remaining: 4m 2s
89:	total: 5m 57s	remaining: 3m 58s
90:	total: 6m	remaining: 3m 53s
91:	total: 6m 4s	remaining: 3m 49s
92:	total: 6m 7s	remaining: 3m 45s
93:	total: 6m 12s	remaining: 3m 41s
94:	total: 6m 17s	remaining: 3m 38s
95:	total: 6m 22s	remaining: 3m 34s
96:	total: 6m 25s	remaining: 3m 30s
97:	total: 6m 29s	remaining: 3m 26s
98:	total: 6m 32s	remaining: 3m 22s
99:	total: 6m 36s	remaining: 3m 18s
100:	total: 6m 41s	remaining: 3m 14s
101:	total: 6m 46s	remaining: 3m 11s
102:	total: 6m 50s	remaining: 3m 7s
103:	total: 6m 54s	remaining: 3m 3s
104:	total: 6m 57s	remaining: 2m 58s
105:	total: 7m	remaining: 2m 54s
106:	total: 7m 3s	remaining: 2m 50s
107:	total: 7m 8s	remaining: 2m 46s
108:	total: 7m 11s	remaining: 2m 42s
109:	total: 7m 14s	remaining: 2m 37

In [92]:
predictions = []
for _model in tqdm.tqdm_notebook(models):
    predictions.append(_model.predict_proba(test)[:,1])
    
predictions = np.vstack(predictions).T.mean(axis=1)

In [ ]:
#prediction = np.hstack([predictions, predictions2]).mean(axis=1)

### XGBoost

In [ ]:
import xgboost as xgb
from xgboost.sklearn import XGBClassifier

In [ ]:
%%time
param_grid1 = {'max_depth':[10],
               'min_child_weight':[2]}

model_xgb = XGBClassifier(learning_rate=0.1, 
                      n_estimators=200, 
                      max_depth=10,
                      min_child_weight=1, 
                      gamma=0, 
                      subsample=0.8, 
                      colsample_bytree=0.8,
                      objective= 'binary:logistic', 
                      nthread=7, 
                      scale_pos_weight=1, 
                      eval_metric='auc',
                      seed=29)

In [ ]:
model_xgb.fit(X_train_train, y_train_train, eval_set=[(X_train_val, y_train_val)])

## LightGBM

In [ ]:
import lightgbm

In [ ]:
model_gbm = lightgbm.LGBMClassifier(
    seed=np.random.randint(10**10),
    n_estimators=500,
    max_depth=10,
    num_leaves=200,
    min_data_in_leaf=100,
    subsample=0.8,
    colsample_bytree=0.8,
    #is_unbalance=True,
    objective='binary'
)

In [ ]:
model_gbm.fit(X_train_train, y_train_train, 
              eval_set=[(X_train_val, y_train_val)], 
              eval_metric='auc', 
              categorical_feature=categorical_features_indices)